In [424]:
import requests
import json
import pandas as pd
import numpy as np

In [425]:
sg_df = pd.read_csv('SG_car_cleaned.csv')

In [427]:
def train_test_split(df, percent_split):
  """split the df into train and test dataframe with
  the number of data in train and test will be equals to:
  train_df: df[:int(n*percent_split)]
  test_df: df[int(percent_split*n):]

  Parameters
    df [pandas DataFrame] -- the dataset that we want to split
    percent_split [int in range [0,1]] -- percentage of data we want to use for training
  """
  df = df.reindex(np.random.permutation(df.index)) # shuffle the examples
  split = int(len(df) * percent_split)
  train_df = df[:split]
  test_df = df[split:]
  return train_df, test_df

percent_split = 0.40
test_df, _ = train_test_split(sg_df, percent_split)

print(len(test_df))

11016


In [428]:
test_df = test_df.drop(columns='timestamp', axis=1)

In [429]:
test_df

,Unnamed: 0,Unnamed: 0.1,trj_id,osname,pingtimestamp,latitude,longitude,speed,bearing,accuracy,...,sin_hour_of_day,cos_hour_of_day,sin_day_of_week,cos_day_of_week,haversine_km,haversine_m,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1
5114,5201,5646821,24942,android,1554782103,1.329601,103.708279,16.207272,173,3.900,...,7.071068e-01,0.707107,0.781831,0.623490,16.278794,10.114483,0.128873,-0.030999,-0.015330,-0.056992
26584,27022,29268585,81845,ios,1555150535,1.338184,103.901839,8.419806,65,8.000,...,5.000000e-01,-0.866025,-0.974928,-0.222521,11.302330,7.022462,-0.063106,-0.004862,0.023191,-0.058729
9443,9609,10439183,41476,android,1555499328,1.295481,103.851432,1.239772,224,3.080,...,2.588190e-01,-0.965926,0.974928,-0.222521,6.971643,4.331682,-0.016789,-0.051969,0.045559,-0.044854
6595,6703,7274446,30300,android,1554857190,1.319879,103.890775,13.140000,70,9.648,...,0.000000e+00,1.000000,0.974928,-0.222521,12.076064,7.503206,-0.053751,-0.024097,0.048691,-0.060438
16628,16896,18326438,64630,android,1555389718,1.303477,103.858433,12.536667,226,3.900,...,8.660254e-01,0.500000,0.781831,0.623490,7.468469,4.640374,-0.023034,-0.043370,0.044179,-0.044656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15077,15324,16628312,61857,android,1555488782,1.306037,103.849002,0.000000,30,3.000,...,8.660254e-01,-0.500000,0.974928,-0.222521,8.369519,5.200223,-0.013410,-0.041678,-0.051318,0.023411
12771,12988,14144618,58781,ios,1554987009,1.321600,103.969775,23.637817,214,12.000,...,1.224647e-16,-1.000000,0.433884,-0.900969,20.171545,12.533160,-0.132268,-0.015204,0.046247,-0.048360
17813,18102,19626763,66711,ios,1555056622,1.404250,103.744863,18.317631,38,8.000,...,8.660254e-01,-0.500000,-0.433884,-0.900969,9.676326,6.012179,0.099224,0.046666,0.012161,0.044058
8369,8509,9237652,37263,android,1555029381,1.342872,103.692204,0.000000,0,3.000,...,0.000000e+00,1.000000,-0.433884,-0.900969,14.272888,8.868155,0.146087,-0.019244,0.017614,-0.019914


In [430]:
test_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'trj_id', 'osname', 'pingtimestamp',
       'latitude', 'longitude', 'speed', 'bearing', 'accuracy', 'travel_time',
       'pickup_time', 'dropoff_time', 'latitude_origin', 'longitude_origin',
       'latitude_destination', 'longitude_destination', 'hour_of_day',
       'day_of_week', 'is_weekend', 'is_weekday', 'is_wee_hours',
       'is_rush_hours_morning', 'is_rush_hours_evening', 'sin_hour_of_day',
       'cos_hour_of_day', 'sin_day_of_week', 'cos_day_of_week', 'haversine_km',
       'haversine_m', 'pickup_pca0', 'pickup_pca1', 'dropoff_pca0',
       'dropoff_pca1'],
      dtype='object')

In [431]:
test_df.columns = ['Unnamed: 0', 'Unnamed: 0.1', 'trj_id', 'osname', 'timestamp',
       'latitude', 'longitude', 'speed', 'bearing', 'accuracy', 'travel_time',
       'pickup_time', 'dropoff_time', 'latitude_origin', 'longitude_origin',
       'latitude_destination', 'longitude_destination', 'hour_of_day',
       'day_of_week', 'is_weekend', 'is_weekday', 'is_wee_hours',
       'is_rush_hours_morning', 'is_rush_hours_evening', 'sin_hour_of_day',
       'cos_hour_of_day', 'sin_day_of_week', 'cos_day_of_week', 'haversine_km',
       'haversine_m', 'pickup_pca0', 'pickup_pca1', 'dropoff_pca0',
       'dropoff_pca1']

In [432]:
test_df

,Unnamed: 0,Unnamed: 0.1,trj_id,osname,timestamp,latitude,longitude,speed,bearing,accuracy,...,sin_hour_of_day,cos_hour_of_day,sin_day_of_week,cos_day_of_week,haversine_km,haversine_m,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1
5114,5201,5646821,24942,android,1554782103,1.329601,103.708279,16.207272,173,3.900,...,7.071068e-01,0.707107,0.781831,0.623490,16.278794,10.114483,0.128873,-0.030999,-0.015330,-0.056992
26584,27022,29268585,81845,ios,1555150535,1.338184,103.901839,8.419806,65,8.000,...,5.000000e-01,-0.866025,-0.974928,-0.222521,11.302330,7.022462,-0.063106,-0.004862,0.023191,-0.058729
9443,9609,10439183,41476,android,1555499328,1.295481,103.851432,1.239772,224,3.080,...,2.588190e-01,-0.965926,0.974928,-0.222521,6.971643,4.331682,-0.016789,-0.051969,0.045559,-0.044854
6595,6703,7274446,30300,android,1554857190,1.319879,103.890775,13.140000,70,9.648,...,0.000000e+00,1.000000,0.974928,-0.222521,12.076064,7.503206,-0.053751,-0.024097,0.048691,-0.060438
16628,16896,18326438,64630,android,1555389718,1.303477,103.858433,12.536667,226,3.900,...,8.660254e-01,0.500000,0.781831,0.623490,7.468469,4.640374,-0.023034,-0.043370,0.044179,-0.044656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15077,15324,16628312,61857,android,1555488782,1.306037,103.849002,0.000000,30,3.000,...,8.660254e-01,-0.500000,0.974928,-0.222521,8.369519,5.200223,-0.013410,-0.041678,-0.051318,0.023411
12771,12988,14144618,58781,ios,1554987009,1.321600,103.969775,23.637817,214,12.000,...,1.224647e-16,-1.000000,0.433884,-0.900969,20.171545,12.533160,-0.132268,-0.015204,0.046247,-0.048360
17813,18102,19626763,66711,ios,1555056622,1.404250,103.744863,18.317631,38,8.000,...,8.660254e-01,-0.500000,-0.433884,-0.900969,9.676326,6.012179,0.099224,0.046666,0.012161,0.044058
8369,8509,9237652,37263,android,1555029381,1.342872,103.692204,0.000000,0,3.000,...,0.000000e+00,1.000000,-0.433884,-0.900969,14.272888,8.868155,0.146087,-0.019244,0.017614,-0.019914


In [433]:
def df_to_model_input(df):
    df_test = df[["latitude_origin", "longitude_origin", "latitude_destination","longitude_destination", "timestamp", "hour_of_day", "day_of_week"]]
    input_list = []
    for index, row in df_test.iterrows():
        row = list(row)
        input_list.append({key:value for key, value in zip(df_test.columns, row)})
#         input_list.append(row)
    return input_list
input_list = df_to_model_input(test_df)

In [434]:
input_list

[{'latitude_origin': 1.3296012,
  'longitude_origin': 103.708279,
  'latitude_destination': 1.2906115,
  'longitude_destination': 103.8495226,
  'timestamp': 1554782103.0,
  'hour_of_day': 3.0,
  'day_of_week': 1.0},
 {'latitude_origin': 1.3381842482018913,
  'longitude_origin': 103.90183872515043,
  'latitude_destination': 1.29238190428477,
  'longitude_destination': 103.81100348054987,
  'timestamp': 1555150535.0,
  'hour_of_day': 10.0,
  'day_of_week': 5.0},
 {'latitude_origin': 1.2954811000000002,
  'longitude_origin': 103.8514324,
  'latitude_destination': 1.3082325,
  'longitude_destination': 103.7899892,
  'timestamp': 1555499328.0,
  'hour_of_day': 11.0,
  'day_of_week': 2.0},
 {'latitude_origin': 1.3198787,
  'longitude_origin': 103.8907745,
  'latitude_destination': 1.2929976,
  'longitude_destination': 103.7854534,
  'timestamp': 1554857190.0,
  'hour_of_day': 0.0,
  'day_of_week': 2.0},
 {'latitude_origin': 1.3034773000000002,
  'longitude_origin': 103.8584332,
  'latitude_

In [435]:
def list_to_json(input_list):
    return json.dumps({'data': input_list})

In [436]:
input_list = df_to_model_input(test_df)
print(input_list)
test_samples = json.dumps({'data': 
    input_list
})
print(test_samples)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [437]:
print(len(input_list))

11016


In [438]:
def call_model_prediction(input_json):
    '''call prediction of input data as json, send over to api'''
    response = requests.post(
        'http://12929100-4b3e-48aa-b7bc-329316a625ee.eastus.azurecontainer.io/score',
        data=input_json, headers=headers)
    # print(response.status_code)
    # print(response.elapsed)
    predictions = response.json()
    return predictions

In [439]:
predictions = call_model_prediction(test_samples)

In [440]:
len(predictions)

11016

In [443]:
from sklearn.metrics import mean_squared_error

In [444]:
y_test = test_df
y_test = y_test['travel_time']
y_test.shape

(11016,)

In [445]:
mse = mean_squared_error(y_test,np.array(predictions))

In [446]:
rmse = np.sqrt(mse)

In [447]:
rmse

278.2664019244917